In [5]:
import time
zeitanfang = time.time()

In [6]:
import pandas as pd
df = pd.read_csv("Exampledataframe.csv", sep=';', header=1)
df.drop(index=df.index[0], 
        axis=0, 
        inplace=True)
df = df.rename(columns={'answer_nps_comment': 'Review'})

# Pre Processing

In [7]:
df.drop(['interview_id', 'answer_nps'], axis=1, inplace=True)

In [ ]:
#Create List of reviews
train_text_list = df['Review'].tolist()
print(train_text_list)

In [ ]:
#Tokenize the sentences
!pip3 install nltk
import nltk
from nltk import word_tokenize
from nltk import StanfordTagger
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

tagged_list = []
for i in train_text_list:
    tagged_text_list = nltk.word_tokenize(i)
    pos_tagged = nltk.pos_tag(tagged_text_list)
    tagged_list.append(pos_tagged)


print(tagged_list)

In [ ]:
#Filter the word with tag- noun,adjective,verb,adverb, number
def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['CD','NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list

#train list after filter
final_tagged_list=filterTag(tagged_list)
print(final_tagged_list)

In [ ]:
data={'Review':final_tagged_list}
data = pd.DataFrame(data)
data.index = range(1,len(data)+1)
print(data)
df["Review"] = data["Review"]
df.head(5)

# Training Aspect Extraction model

In [13]:
from sklearn.model_selection import train_test_split

X= df.Review
y = df.drop('Review',1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


#Only for Bagging
#df_train = X_train.to_frame().merge(y_train, how='inner', left_index=True, right_index=True)
#df_sample_train = df_train.sample(frac=1, replace=True, random_state=40)


#X_train = df_sample_train.Review
#y_train = df_sample_train.drop('Review',1)

y_train = y_train.fillna(10)
y_train = y_train.astype(str).astype(int)
y_train[y_train <= 0] = 1
y_train[y_train == 10] = 0

y_test = y_test.fillna(10)
y_test = y_test.astype(str).astype(int)
y_test[y_test <= 0] = 1
y_test[y_test == 10] = 0


X_train = X_train.astype(str)
X_test = X_test.astype(str)

y_train_aspect = y_train.fillna(10)
y_train_aspect = y_train_aspect.astype(str).astype(int)
y_train_aspect[y_train_aspect <= 0] = 1
y_train_aspect[y_train_aspect == 10] = 0

y_test_aspect = y_test.fillna(10)
y_test_aspect = y_test_aspect.astype(str).astype(int)
y_test_aspect[y_test_aspect <= 0] = 1
y_test_aspect[y_test_aspect == 10] = 0

import numpy as np

y_train_aspect = np.asarray(y_train_aspect, dtype=np.int64)
y_test_aspect = np.asarray(y_test_aspect, dtype=np.int64)

X_train = X_train.tolist()
X_test = X_test.tolist()

<ipython-input-13-1006e2453262>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  y = df.drop('Review',1)
<ipython-input-13-1006e2453262>:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  y_train = df_sample_train.drop('Review',1)


In [14]:
X2 = data

X = []
sentences = list(data["Review"])
for sen in sentences:
    X.append(sen)

In [15]:
Z = df.drop('Review',1)
Z = Z.fillna(10)
Z = Z.astype(str).astype(int)
Z[Z <= 0] = 1
Z[Z == 10] = 0
y = Z

<ipython-input-15-a023255a5894>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Z = df.drop('Review',1)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [19]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [20]:
#!unzip glove*.zip

In [21]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('glove.6B.100d.txt', encoding='utf-8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [22]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM, Conv2D, SpatialDropout1D, GlobalMaxPool1D
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
import tensorflow.keras.optimizers 
from tensorflow.keras.optimizers import SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint



import pandas as pd
import numpy as np
import re


In [23]:
from tensorflow.keras.layers import Dense, Input, LSTM, SimpleRNN, Embedding, Dropout, SpatialDropout1D, Activation, Conv1D,GRU
from tensorflow.keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization, Add, Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D

callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(monitor="loss", mode="min", patience=6, restore_best_weights=True)
]

maxlen = 200
sequence_input = Input(shape=(maxlen, ))
model = Embedding(input_dim =2618, input_length=maxlen, output_dim=100,weights=[embedding_matrix], trainable=False)(sequence_input)
model = SpatialDropout1D(0.5)(model)
model = GRU(25, return_sequences=True,dropout=0.1)(model)
model = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(model)
avg_pool = GlobalAveragePooling1D()(model)
max_pool = GlobalMaxPooling1D()(model)
model = concatenate([avg_pool, max_pool]) 
preds = Dense(14, activation="sigmoid")(model)
GRU_Glove_model = Model(sequence_input, preds)
GRU_Glove_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])


model = GRU_Glove_model.fit(X_train, y_train, batch_size=1, epochs=100, validation_data=(X_test, y_test), callbacks=[callbacks])

Epoch 1/100
1251/1251 [==============================] - 24s 16ms/step - loss: 0.3082 - accuracy: 0.5619 - val_loss: 0.2277 - val_accuracy: 0.6403
Epoch 2/100
1251/1251 [==============================] - 19s 15ms/step - loss: 0.2282 - accuracy: 0.5808 - val_loss: 0.2210 - val_accuracy: 0.6331
Epoch 3/100
1251/1251 [==============================] - 19s 15ms/step - loss: 0.2108 - accuracy: 0.6318 - val_loss: 0.2016 - val_accuracy: 0.6403
Epoch 4/100
1251/1251 [==============================] - 19s 15ms/step - loss: 0.1962 - accuracy: 0.6508 - val_loss: 0.2035 - val_accuracy: 0.6115
Epoch 5/100
1251/1251 [==============================] - 19s 15ms/step - loss: 0.1941 - accuracy: 0.6431 - val_loss: 0.1967 - val_accuracy: 0.6331
Epoch 6/100
1251/1251 [==============================] - 19s 15ms/step - loss: 0.1841 - accuracy: 0.6417 - val_loss: 0.1931 - val_accuracy: 0.6475
Epoch 7/100
1251/1251 [==============================] - 19s 15ms/step - loss: 0.1782 - accuracy: 0.6342 - val_loss: 0

In [ ]:
score = GRU_Glove_model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [25]:
predictions = GRU_Glove_model.predict(X_test)

In [26]:
predictions2 = predictions.tolist()
data2={'GRU_Predictions':predictions2}
data2 = pd.DataFrame(data2)

In [27]:
print(data2)

                                       GRU_Predictions
0    [0.946509838104248, 0.5664821267127991, 0.2368...
1    [0.8784558176994324, 0.049329593777656555, 0.0...
2    [0.8220332860946655, 0.3336625099182129, 0.142...
3    [0.4872305691242218, 0.20868344604969025, 0.00...
4    [0.953498363494873, 0.222971573472023, 0.00533...
..                                                 ...
134  [0.2277623862028122, 0.020373506471514702, 0.8...
135  [0.9516741633415222, 0.06668079644441605, 0.94...
136  [0.4619810879230499, 0.7922795414924622, 0.107...
137  [0.9393295049667358, 0.19237197935581207, 0.02...
138  [0.984899640083313, 0.030612053349614143, 0.00...

[139 rows x 1 columns]


In [28]:
data2.to_csv('ac/gru_predictions.csv', index=False)  
#data2.to_csv('bagging/ac/gru_predictions.csv', index=False)  

In [ ]:
zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

In [ ]:
#Boosting
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.multiclass import OneVsRestClassifier
import time
zeitanfang = time.time()


def simple_model():  
    maxlen = 200
    sequence_input = Input(shape=(maxlen, ))
    model = Embedding(input_dim =2618, input_length=maxlen, output_dim=100,weights=[embedding_matrix], trainable=False)(sequence_input)
    model = SpatialDropout1D(0.5)(model)
    model = GRU(25, return_sequences=True,dropout=0.1)(model)
    model = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(model)
    avg_pool = GlobalAveragePooling1D()(model)
    max_pool = GlobalMaxPooling1D()(model)
    model = concatenate([avg_pool, max_pool]) 
    preds = Dense(1, activation="sigmoid")(model)
    GRU_Glove_model = Model(sequence_input, preds)
    GRU_Glove_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])
    return GRU_Glove_model


ann_estimator = KerasRegressor(build_fn=simple_model, epochs=10, batch_size=25, verbose=1)


boosted_ann = OneVsRestClassifier(AdaBoostRegressor(base_estimator= ann_estimator, n_estimators = 15))
boosted_ann.fit(X_train, y_train)
predictions = boosted_ann.predict(X_test)

zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

In [27]:
predictions2 = predictions.tolist()
data2={'GRU':predictions2}
data2 = pd.DataFrame(data2)

In [28]:
data2.to_csv('boosting/gru_boosting.csv', index=False)  

# Training Aspect-based Sentiment Analysis model

In [29]:
#Create binary datasets for positive, negative and neutral sentiments
#Positive Dataset
X= df.Review
y = df.drop('Review',1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


#Only for Bagging
#df_train = X_train.to_frame().merge(y_train, how='inner', left_index=True, right_index=True)
#df_sample_train = df_train.sample(frac=1, replace=True, random_state=40)


#X_train = df_sample_train.Review
#y_train = df_sample_train.drop('Review',1)


#Train Dataset
df_positive_train_X = X_train
df_positive_train_y = y_train


df_positive_train_y = df_positive_train_y.fillna(0)
df_positive_train_y = df_positive_train_y.astype(str).astype(int)
df_positive_train_y[df_positive_train_y <= 0] = 0

#df_positive_train = df_positive_train_X.to_frame().merge(df_positive_train_y, how='inner', left_index=True, right_index=True)

df_positive_train_y["Review"] = df_positive_train_X
df_positive_train = df_positive_train_y

#Test Dataset
df_positive_test_X = X_test
df_positive_test_y = y_test

df_positive_test_y = df_positive_test_y.fillna(0)
df_positive_test_y = df_positive_test_y.astype(str).astype(int)
df_positive_test_y[df_positive_test_y <= 0] = 0

#df_positive_test = df_positive_test_X.to_frame().merge(df_positive_test_y, how='inner', left_index=True, right_index=True)

df_positive_test_y["Review"] = df_positive_test_X
df_positive_test = df_positive_test_y

<ipython-input-29-3e4fa491b959>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  y = df.drop('Review',1)


In [ ]:
#Get all aspects in one list
all_aspects = []
for col in y.columns:
    all_aspects.append(col)
print(all_aspects)

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
vect = CountVectorizer(max_df=1.0,stop_words='english')  
X_train_dtm = vect.fit_transform(X_train.astype(str))
X_test_dtm = vect.fit_transform(X_test.astype(str))

In [32]:
#Generating extra feature that indicates which aspect category is present in the review
def get_dict_aspect(y,all_aspects):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(all_aspects)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect



from sklearn.feature_extraction import DictVectorizer

#Creates a dictonionary of all aspects and a binary classifier which indicates if a aspect occour in a sentence.
#y_train_aspect = np.asarray(y_train_aspect, dtype=np.int64)
train_dict_aspect=get_dict_aspect(y_train_aspect, all_aspects)
#Declare DictVectorizer
d_train=DictVectorizer()
X_train_aspect_dtm = d_train.fit_transform(train_dict_aspect)


#The same for the test dataset
test_dict_aspect=get_dict_aspect(y_test_aspect, all_aspects)
d_test=DictVectorizer()
X_test_aspect_dtm = d_test.fit_transform(test_dict_aspect)

In [33]:
#Function to classify a sentiment (Used for positive, negative)
from sklearn.feature_extraction.text import CountVectorizer

def classify_sentiment(df_train,df_test,X_train_aspect_dtm,X_test_aspect_dtm):
    
    import numpy as np
    X_train = df_train.Review
    y_train = df_train.drop('Review',1)
    y_train = np.asarray(y_train, dtype=np.int64)

    X_test = df_test.Review
    y_test = df_test.drop('Review',1)
    y_test = np.asarray(y_test, dtype=np.int64)
    
    #convert sentences to vectors in order to be able to process them 
    vect_sen = CountVectorizer(stop_words='english',ngram_range=(1,2))  
    X_train_dtm = vect_sen.fit_transform(X_train)
    X_test_dtm = vect_sen.transform(X_test.astype(str))

    #ombining word vector with extra feature. (Adds list of aspecs in sentences)
    from scipy.sparse import hstack
    X_train_dtm=hstack((X_train_dtm, X_train_aspect_dtm))
    X_test_dtm=hstack((X_test_dtm, X_test_aspect_dtm))

    from tensorflow.keras.layers import Dense, Input, LSTM, SimpleRNN, Embedding, Dropout, SpatialDropout1D, Activation, Conv1D,GRU
    from tensorflow.keras.layers import Conv1D, Bidirectional, GlobalMaxPool1D, MaxPooling1D, BatchNormalization, Add, Flatten
    from tensorflow.keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D

    filter_length = 300

    early = EarlyStopping(monitor="val_loss", mode="min", patience=4)
    maxlen = 8628
    sequence_input = Input(shape=(maxlen, ))
    model = Embedding(input_dim =2618, input_length=maxlen, output_dim=100,weights=[embedding_matrix], trainable=False)(sequence_input)
    model = SpatialDropout1D(0.5)(model)
    model = GRU(25, return_sequences=True,dropout=0.1)(model)
    model = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(model)
    avg_pool = GlobalAveragePooling1D()(model)
    max_pool = GlobalMaxPooling1D()(model)
    model = concatenate([avg_pool, max_pool]) 
    preds = Dense(14, activation="sigmoid")(model)
    
    GRU_Glove_model = Model(sequence_input, preds)
    
    GRU_Glove_model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=['accuracy'])

    X_train_dtm = pd.DataFrame.sparse.from_spmatrix(X_train_dtm)

    X_test_dtm = pd.DataFrame.sparse.from_spmatrix(X_test_dtm)    

    callbacks = [
        ReduceLROnPlateau(),
        EarlyStopping(patience=6),
        ModelCheckpoint(filepath='model-lstm.h5', save_best_only=True)
    ]

    GRU_Glove_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

    history = GRU_Glove_model.fit(X_train_dtm, y_train,
                    epochs=100,
                    batch_size=1,
                    verbose=1,
                    validation_data=(X_test_dtm, y_test),
                    callbacks=callbacks)

    predictions2 = GRU_Glove_model.predict(X_test_dtm)

    return (predictions2)

In [ ]:
#For positive sentiment classifier
import time
zeitanfang = time.time()

predictions=classify_sentiment(df_positive_train,df_positive_test,X_train_aspect_dtm,X_test_aspect_dtm)

predictions = predictions.tolist()
data2={'lstm_Predictions':predictions}
data2 = pd.DataFrame(data2)
data2.to_csv('asa/gru_predictions.csv', index=False)  
#data2.to_csv('bagging/asa/gru_predictions.csv', index=False)  

zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)